<p style="font-size: 27px;"> 🎬 <b>Project 3: Sentiment analysis with transformer</b></p>
<p style="font-size: 16px;"> In this project, we would like to do sentiment analysis on a large movie review dataset IMDB, which contains movie reviews and binary sentiment score (1 indicates a positive review and 0 indicates a negative review). We want to use a simple transformer to learn from the data, and predict the sentiment given our own movie review! </p>

<p style="font-size: 16px;"> Submission note: We grade the content in this notebook. Make sure outputs are present. You will package everything in a zip file and submit it with the following format:
f"P3_{LastName}_{FirstName}.V{version_number}.zip", e.g. "P3_Smit_John_V1.zip". </p>

<p style="font-size: 16px;"> For the questions in the comments, there is no need to write down the answers in the notebook. You just need to prepare the answers for the project interview. Please note that the interview will cover not just the comment questions, but also your overall understanding of the project and the techniques used. Make sure you understand what each step of the code is doing.</p>

<p style="font-size: 25px;"> Task 0: Setup environment</p>
<p style="font-size: 16px;"> Please install the necessary packages in your environment and import the following libraries. </p>
<p style="font-size: 16px;"> The packages we will need: pytorch, numpy, tqdm, transformers and datasets. </p>

In [ ]:
import datasets
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tqdm
import transformers

In [ ]:
seed = 0

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')  # You can switch to 'cpu' for debugging

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

<p style="font-size: 25px;"> Task 1: Load dataset (5pt)</p>
<p style="font-size: 16px;"> Now we need to load the IMDB dataset and setup the dataloader. </p>

In [3]:
train_data, test_data = datasets.load_dataset("imdb", split=["train", "test"])

In [ ]:
# Let's have a look at one example from the dataset
train_data[0]
# How many samples are there in the training dataset?

In [ ]:
# In the raw dataset, the labels are strings. We need to use tokenizer to convert them to integers.
tokenizer_name = "bert-base-uncased"
tokenizer = transformers.AutoTokenizer.from_pretrained(tokenizer_name)

In [ ]:
# What is stored in the tokenizer? What the printed information / parameters means?
tokenizer

In [ ]:
def tokenize_and_numericalize_example(example, tokenizer):
    ids = tokenizer(example["text"], truncation=True)["input_ids"]
    return {"ids": ids}

train_data = train_data.map(
    tokenize_and_numericalize_example, fn_kwargs={"tokenizer": tokenizer}
)
test_data = test_data.map(
    tokenize_and_numericalize_example, fn_kwargs={"tokenizer": tokenizer}
)

In [ ]:
# What does the processed training data look like?
train_data[0]

In [7]:
valid_size = 0.25

train_valid_data = train_data.train_test_split(test_size=valid_size)
train_data = train_valid_data["train"]
valid_data = train_valid_data["test"]

# We need to convert column 'ids' and 'label' to torch format
train_data = train_data.with_format(type="torch", columns=["ids", "label"])  
valid_data = valid_data.with_format(type="torch", columns=["ids", "label"])
test_data = test_data.with_format(type="torch", columns=["ids", "label"])

def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_ids = [i["ids"] for i in batch]
        batch_ids = nn.utils.rnn.pad_sequence(
            batch_ids, padding_value=pad_index, batch_first=True
        )
        batch_label = [i["label"] for i in batch]
        batch_label = torch.stack(batch_label)
        batch = {"ids": batch_ids, "label": batch_label}
        return batch

    return collate_fn

def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = # TODO: Please define the DataLoader here. You need to use the collate_fn and set shuffle.
    return data_loader

In [8]:
batch_size = # TODO: Please set a proper batch size
pad_index = tokenizer.pad_token_id
train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

<p style="font-size: 25px;"> Task 2: Implement the transformer (55pt)</p>
<p style="font-size: 16px;"> Now it's time to implement a basic transformer by ourselves. We will focus on implementing the ⚡attention⚡ mechanism.</p>

In [9]:
class Embedding(nn.Module):
    def __init__(self, vocab_size, max_length, embed_dim, dropout=0.1):
        super(Embedding, self).__init__()
        self.word_embed = # TODO: Please define the word embedding layer using vocab_size and embed_dim.
        self.pos_embed = # TODO: Please define the positional embedding layer using max_length and embed_dim.
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        batch_size, seq_length = x.shape
        positions = # TODO: Fill in the blanks. Hint: positions = torch.arange(0, ___).expand(batch_size, ___).to(device)
        embedding = # TODO: Compute the final embedding.
        return self.dropout(embedding)

In [ ]:
def scaled_dot_product_attention(Q, K, V, mask=None):
    d_k = # TODO: Key dimension
    scores = # TODO: Please compute the attention scores

    # Why the masking is implemented in this way?
    if mask is not None:
        scores = scores.masked_fill(mask == 0, float('-inf'))

    attn_weights = F.softmax(scores, dim=-1)
    output = # TODO: Please compute the attention output
    
    return output, attn_weights

class MultiHeadAttention(torch.nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        assert embed_dim % num_heads == 0  # Ensure the number of heads divides evenly into embedding dimension
        self.head_dim = # TODO: Please compute the head dimension

        # Linear layers to project the queries, keys, and values
        # Please define the linear layers for W_q, W_k, and W_v using embed_dim
        self.W_q = # TODO: 
        self.W_k = # TODO: 
        self.W_v = # TODO: 

        # Final output projection
        # Please define the linear layer for W_o using embed_dim
        self.W_o = # TODO: 

    def forward(self, Q, K, V, mask=None):
        batch_size = Q.size(0)

        # Step 1: Project Q, K, V using the linear layers
        # Please fill in the blanks. What should be the shape of Q, K, and V after the linear layers?
        Q = # TODO:
        K = # TODO:
        V = # TODO:

        # Step 2: Split the projections into multiple heads
        # Please fill in the blanks. What should be the shape of Q, K, and V after the split?
        Q = # TODO:
        K = # TODO:
        V = # TODO:

        # Step 3: Apply scaled dot-product attention
        attn_output, attn_weights = scaled_dot_product_attention(Q, K, V, mask)

        # Step 4: Concatenate the attention output from all heads
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, -1, self.embed_dim)  # (batch_size, seq_len, embed_dim)

        # Step 5: Project the concatenated output back to the original embedding dimension
        # Please fill in the blanks. What should be the shape of attn_output after the linear layer?
        output = # TODO:

        return output

In [11]:
# What does the transformer encoder look like?
# You can modify the architecture of the encoder as you need.
class TransformerEncoder(nn.Module):
    def __init__(self, embed_dim, num_heads, forward_expansion, dropout):
        super(TransformerEncoder, self).__init__()

        self.attention = MultiHeadAttention(embed_dim, num_heads)
        self.norm1 = # TODO: Please define a LayerNorm layer
        self.norm2 = # TODO: Please define a LayerNorm layer

        self.feed_forward = # TODO: Please define a feed-forward network here with at least two nn.Linear layers.
        # Please use forward_expansion * embed_dim as the hidden layer dimension.
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        attention_out = # TODO: Please compute the output of the multi-head attention layer using self.attention and self.dropout.
        x =  # TODO: Please apply the residual connection and normalization. 
        forward_out = self.dropout(self.feed_forward(x))
        out =  # TODO: Please apply the residual connection and normalization. 

        return out

In [ ]:
class Transformer(nn.Module):
    def __init__(self, vocab_size, max_length, embed_dim,
                num_heads, forward_expansion, dropout):
        super(Transformer, self).__init__()

        self.embedder = Embedding(vocab_size, max_length, embed_dim)
        self.encoder = TransformerEncoder(embed_dim, num_heads, forward_expansion, dropout)
        self.fc = # TODO: Please define the final fully connected layer.

    def forward(self, x):
        embedding = self.embedder(x)
        encoding = # TODO: Compute the encoding using encoder

        # Is the max pooling a good choice here? Why? Or what should be used instead?
        compact_encoding = encoding.max(dim=1)[0]

        out = self.fc(compact_encoding)
        return out

<p style="font-size: 25px;"> Task 3: Prepare training loop (25pt)</p>
<p style="font-size: 16px;"> In this task, we will setup the training hyperparameters and the training loop.</p>  

In [ ]:
# What are the meanings of the following parameters?
embedding_dim = # TODO: Please set the embedding dimension
num_heads = # TODO: Please set the number of heads
dropout = # TODO: Please set a dropout dropout rate
forward_expansion = 3
max_length = 512
vocab_size = 35000

model = Transformer(
    vocab_size, max_length, embedding_dim, num_heads, forward_expansion, dropout)
model = model.to(device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

In [61]:
lr = # TODO: Please set a proper learning rate
optimizer = # TODO: Please define an optimizer
criterion = # TODO: What loss function should we use?

In [58]:
def binary_accuracy(preds, labels):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct_preds = (rounded_preds == labels).float()
    acc = # TODO: accuracy should be a ratio of correct predictions to total predictions
    return acc

In [56]:
def train(data_loader, model, criterion, optimizer, device):
    model.train()
    epoch_losses = []
    epoch_accs = []
    for batch in tqdm.tqdm(data_loader, desc="training..."):
        optimizer.zero_grad()
        ids = batch["ids"].to(device)
        label = batch["label"].to(device)
        # TODO: Please complete the training loop
        loss = # Please compute the loss
        accuracy = # Please use binary_accuracy function to compute the accuracy
        epoch_losses.append(loss.item())
        epoch_accs.append(accuracy.item())
    return np.mean(epoch_losses), np.mean(epoch_accs)

In [57]:
def evaluate(data_loader, model, criterion, device):
    model.eval()
    epoch_losses = []
    epoch_accs = []
    with torch.no_grad():
        for batch in tqdm.tqdm(data_loader, desc="evaluating..."):
            ids = batch["ids"].to(device)
            label = batch["label"].to(device)
            # TODO: Please complete the evaluation loop. You need to compute the loss and accuracy
            loss = # Please compute the loss
            accuracy = # Please use binary_accuracy function to compute the accuracy
            epoch_losses.append(loss.item())
            epoch_accs.append(accuracy.item())
    return np.mean(epoch_losses), np.mean(epoch_accs)

<p style="font-size: 25px;">Task 4: Start training (10pt)</p>
<p style="font-size: 16px;">Let's run the following code and print the output. We want to make the testing accuracy higher than <b>0.85</b>!</p>

In [ ]:
n_epochs = # TODO: Please set the number of epochs as you need

for epoch in range(n_epochs):
    train_loss, train_acc = train(train_data_loader, model, criterion, optimizer, device)
    valid_loss, valid_acc = evaluate(valid_data_loader, model, criterion, device)
    print(f"epoch: {epoch}")
    print(f"train_loss: {train_loss:.3f}, train_acc: {train_acc:.3f}")
    print(f"valid_loss: {valid_loss:.3f}, valid_acc: {valid_acc:.3f}")

In [ ]:
test_loss, test_acc = evaluate(test_data_loader, model, criterion, device)
print(f"test_loss: {test_loss:.3f}, test_acc: {test_acc:.3f}")

<p style="font-size: 25px;">Task 5: Try out our model 🚀 (5pt)</p>
<p style="font-size: 16px;">Let's try out our model to see if it can correctly classify the sentiment of the input text!</p>

In [ ]:
def predict_sentiment(text, model, tokenizer, device):
    ids = tokenizer(text)["input_ids"]
    tensor = torch.LongTensor(ids).unsqueeze(dim=0).to(device)
    prediction = model(tensor).squeeze()
    predicted_class = torch.round(torch.sigmoid(prediction))
    if predicted_class == 1:
        print("Positive sentiment")
    else:
        print("Negative sentiment")
    return predicted_class

In [ ]:
# TODO: Run the following code to test the model
text = "Definitely love this movie!"
predicted_class = predict_sentiment(text, model, tokenizer, device)

In [ ]:
# TODO: Run the following code to test the model
text = "It is not so terrible but I still don't like it"
predicted_class = predict_sentiment(text, model, tokenizer, device)

In [ ]:
# TODO: Please provide your own movie review and see how the model predicts!
text = # Your own movie review
predicted_class = predict_sentiment(text, model, tokenizer, device)